In [1]:
from tensorflow.python.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM,GRU
 
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
 
from sklearn.preprocessing import MinMaxScaler
 
from datetime import datetime

from binance.client import Client

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

In [2]:
client = Client('', '')

In [3]:
cmc = CoinMarketCapAPI('  ')

In [4]:
info = client.get_all_tickers()

In [5]:
'Good to check indivudual by get_symbol_info if status is trading'

'Good to check indivudual by get_symbol_info if status is trading'

In [6]:
'Filter the Info to get the list of Symbols ending with USDT'

'Filter the Info to get the list of Symbols ending with USDT'

In [7]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith('USDT')]

In [8]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [9]:
target_rank_info = [d['symbol']+'USDT' for d in rank_info if d['rank'] >= 200 and d['rank'] <= 450]

In [10]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [11]:
klines = client.get_historical_klines('BTCBUSD', Client.KLINE_INTERVAL_1HOUR, "30 day ago UTC") 

In [12]:
array = np.array(klines, dtype=np.float64)

In [13]:
focused_kline_array = np.delete(array, [2, 3, 6, 7, 8, 9, 10, 11], 1)

In [14]:
length = len(focused_kline_array)

In [15]:
length

720

In [16]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[2] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value        

In [17]:
percentage_value = find_percentage_increase_or_decrease(focused_kline_array)

In [18]:
percentage_value = np.reshape(percentage_value, (length, 1))

In [19]:
focused_kline_array = np.append(focused_kline_array, percentage_value, 1)

In [20]:
focused_kline_array

array([[ 1.65468960e+12,  3.03927100e+04,  3.01198100e+04,
         1.13645938e+03, -8.97912690e-01],
       [ 1.65469320e+12,  3.01194300e+04,  3.07507800e+04,
         2.57746519e+03,  2.09615521e+00],
       [ 1.65469680e+12,  3.07507800e+04,  3.04099800e+04,
         1.77328795e+03, -1.10826457e+00],
       ...,
       [ 1.65727080e+12,  2.14219700e+04,  2.15584500e+04,
         2.27721380e+03,  6.37102937e-01],
       [ 1.65727440e+12,  2.15558100e+04,  2.15223600e+04,
         1.65034538e+03, -1.55178581e-01],
       [ 1.65727800e+12,  2.15223500e+04,  2.15984700e+04,
         9.27561180e+02,  3.53678850e-01]])

In [21]:
positive_array = focused_kline_array[:,4] >= 0

In [22]:
filtered_positive_array = focused_kline_array[:,4][positive_array]

In [23]:
negative_array = focused_kline_array[:,4] <= 0

In [24]:
filtered_negative_array = focused_kline_array[:,4][negative_array]

In [25]:
average_positive_increase = np.mean(filtered_positive_array)

In [26]:
tolerable_average_positive_increase = average_positive_increase * 0.75

In [27]:
average_negative_increase = np.mean(filtered_negative_array)

In [28]:
tolerable_average_negative_increase = average_negative_increase * 0.75

In [29]:
average_volume = np.mean(focused_kline_array[:,3])

In [30]:
tolerable_volume = average_volume * 0.75

In [31]:
tolerable_volume

1244.1335894895833

In [32]:
tolerable_average_negative_increase

-0.5070151058814183

In [33]:
tolerable_average_positive_increase

0.5164762423146916

In [34]:
def is_candidate(filtered_klines_info):
    red_and_volume_check_klines = filtered_klines_info[0:2]
    green_and_volume_check_klines = filtered_klines_info[2:3]

    is_candidate = False
    for red_and_volume_check_kline in red_and_volume_check_klines:
        if float(red_and_volume_check_kline[4]) <= 0 and float(red_and_volume_check_kline[4]) >= tolerable_average_negative_increase and float(red_and_volume_check_kline[3]) < average_volume:
            is_candidate = True
        else:
            return False
        
    if is_candidate and float(green_and_volume_check_klines[0][4]) > 0 and float(green_and_volume_check_klines[0][4]) <= tolerable_average_positive_increase and float(green_and_volume_check_klines[0][4]) < average_volume:
        is_candidate = True
    else: 
        is_candidate = False
        
        
    return is_candidate    
    

In [35]:
length = int(len(focused_kline_array))

In [36]:
focused_kline_array[62:66]

array([[ 1.65491280e+12,  2.91765300e+04,  2.92377300e+04,
         3.47550010e+02,  2.09757637e-01],
       [ 1.65491640e+12,  2.92377400e+04,  2.92401300e+04,
         2.48836120e+02,  8.17436642e-03],
       [ 1.65492000e+12,  2.92401300e+04,  2.93504200e+04,
         3.19220090e+02,  3.77187106e-01],
       [ 1.65492360e+12,  2.93504200e+04,  2.92543800e+04,
         4.63423550e+02, -3.27218486e-01]])

In [37]:
klines[62:66]

[[1654912800000,
  '29176.53000000',
  '29268.81000000',
  '29155.00000000',
  '29237.73000000',
  '347.55001000',
  1654916399999,
  '10149111.19526540',
  7632,
  '167.07494000',
  '4878566.03468610',
  '0'],
 [1654916400000,
  '29237.74000000',
  '29286.17000000',
  '29202.94000000',
  '29240.13000000',
  '248.83612000',
  1654919999999,
  '7275473.74425550',
  6181,
  '128.99951000',
  '3771589.13216580',
  '0'],
 [1654920000000,
  '29240.13000000',
  '29354.07000000',
  '29221.01000000',
  '29350.42000000',
  '319.22009000',
  1654923599999,
  '9351715.18894810',
  7478,
  '173.56706000',
  '5084896.56369100',
  '0'],
 [1654923600000,
  '29350.42000000',
  '29408.64000000',
  '29223.69000000',
  '29254.38000000',
  '463.42355000',
  1654927199999,
  '13582392.34806000',
  9916,
  '244.08562000',
  '7154780.72045780',
  '0']]

In [38]:
full_test_data_set = []

for x in range(0, length) :
    
    print(x)
    filtered_klines_info = focused_kline_array[x:x+4]
    investment_in_tether = 10000
    if(len(filtered_klines_info)) == 4 and is_candidate(filtered_klines_info):
        print('****** Sucess ****')
        print(x)
        print(filtered_klines_info) 
        key_test_data_set = [] 
        
        buy_kline_info = filtered_klines_info[2:3]
        sell_kline_info = filtered_klines_info[3:4]
        buy_quote = buy_kline_info[0][2]
        sell_quote = sell_kline_info[0][2]
        volume = investment_in_tether/buy_quote
 
        temp = format(sell_kline_info[0][0], '.8f')
               
        key_test_data_set.append(temp)
        
        key_test_data_set.append(buy_quote)
        
        key_test_data_set.append(sell_quote)
        
        key_test_data_set.append(volume)
        
        key_test_data_set.append(investment_in_tether)
        
        dis_investment_in_tether = volume * sell_quote
        key_test_data_set.append(dis_investment_in_tether)
        key_test_data_set.append(dis_investment_in_tether - investment_in_tether)
        full_test_data_set.append(key_test_data_set)

0
1
2
3
4
5
6
****** Sucess ****
6
[[ 1.65471120e+12  3.03626300e+04  3.02091900e+04  1.54031717e+03
  -5.05358067e-01]
 [ 1.65471480e+12  3.02091900e+04  3.01160000e+04  1.01727002e+03
  -3.08482286e-01]
 [ 1.65471840e+12  3.01130000e+04  3.01800000e+04  5.10050110e+02
   2.22495268e-01]
 [ 1.65472200e+12  3.01800000e+04  3.04009600e+04  4.44194330e+02
   7.32140490e-01]]
7
8
9
10
****** Sucess ****
10
[[ 1.65472560e+12  3.04009600e+04  3.02870200e+04  3.93037450e+02
  -3.74790796e-01]
 [ 1.65472920e+12  3.02870200e+04  3.01781500e+04  4.55962140e+02
  -3.59460918e-01]
 [ 1.65473280e+12  3.01781600e+04  3.01992100e+04  6.64120620e+02
   6.97524302e-02]
 [ 1.65473640e+12  3.01992100e+04  3.00665800e+04  4.49348640e+02
  -4.39183674e-01]]
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
****** Sucess ****
33
[[ 1.65480840e+12  3.01599900e+04  3.01120900e+04  3.10746850e+02
  -1.58819681e-01]
 [ 1.65481200e+12  3.01120900e+04  3.00425300e+04  4.94433640e+02
  -2.31003

In [39]:
full_test_data_set = np.array(full_test_data_set)

In [40]:
full_test_data_set

array([['1654722000000.00000000', '30180.0', '30400.96',
        '0.3313452617627568', '10000', '10073.2140490391',
        '73.21404903909934'],
       ['1654736400000.00000000', '30199.21', '30066.58',
        '0.33113448994195543', '10000', '9956.081632598998',
        '-43.91836740100189'],
       ['1654819200000.00000000', '30082.56', '29731.81',
        '0.33241851757297247', '10000', '9883.40420496128',
        '-116.5957950387201'],
       ['1654858800000.00000000', '29976.84', '29996.53',
        '0.33359086548148503', '10000', '10006.56840414133',
        '6.568404141329665'],
       ['1654909200000.00000000', '29206.18', '29176.53',
        '0.34239328799589674', '10000', '9989.848039010922',
        '-10.151960989078361'],
       ['1655035200000.00000000', '27423.09', '27403.32',
        '0.36465620759732037', '10000', '9992.790746775801',
        '-7.209253224198619'],
       ['1656140400000.00000000', '21286.31', '21493.81',
        '0.4697855100296857', '10000', '10097.4

In [41]:
overall_profit = np.sum(full_test_data_set[:, 6].astype(float))

In [42]:
overall_profit

135.25350296811666

In [43]:
df = pd.DataFrame.from_records(full_test_data_set)

In [44]:
df.columns = ['Open Time', 'Buy Quote', 
              'Sell Quote', 'Volume', 'Investment in Tether', 
              'Disinvestment in tether', 'Profit']


In [45]:
df

,Open Time,Buy Quote,Sell Quote,Volume,Investment in Tether,Disinvestment in tether,Profit
0,1654722000000.00000000,30180.0,30400.96,0.3313452617627568,10000,10073.2140490391,73.21404903909934
1,1654736400000.00000000,30199.21,30066.58,0.33113448994195543,10000,9956.081632598998,-43.91836740100189
2,1654819200000.00000000,30082.56,29731.81,0.33241851757297247,10000,9883.40420496128,-116.5957950387201
3,1654858800000.00000000,29976.84,29996.53,0.33359086548148503,10000,10006.56840414133,6.568404141329665
4,1654909200000.00000000,29206.18,29176.53,0.34239328799589674,10000,9989.848039010922,-10.151960989078361
5,1655035200000.00000000,27423.09,27403.32,0.36465620759732037,10000,9992.790746775801,-7.209253224198619
6,1656140400000.00000000,21286.31,21493.81,0.4697855100296857,10000,10097.480493331159,97.48049333115887
7,1656216000000.00000000,21411.25,21408.11,0.46704419405686265,10000,9998.533481230663,-1.466518769337199
8,1656273600000.00000000,21268.56,21388.86,0.47017757666715565,10000,10056.56236247306,56.56236247305969
9,1656424800000.00000000,21035.99,20822.46,0.47537577266389647,10000,9898.493011263077,-101.50698873692272
